In [239]:
import sys 
import numpy as np
import pandas as pd

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
db_path = r"lebron.db"

In [5]:
###############################################################################
#                              Player Functions                               #
###############################################################################

### Available Functions
###
# player_avg_data(player_username)
# player_sum_data(player_username)
# player_per36_data(player_username)
# player_advanced_data(player_username)
# player_season_data(player_username)
###

Lebron has played from 2004-Current (2025)

In [241]:
lebron_season_avgs = scraper.player_avg_data("jamesle01")
lebron_season_per36 = scraper.player_per36_data("jamesle01")
lebron_season_sums = scraper.player_sum_data("jamesle01")
lebron_season_advanced = scraper.player_advanced_data("jamesle01")

---

creating a average data table in the database

In [242]:
COLUMN_RENAME_MAP = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Season Type": "season_type"
}

# Rename columns explicitly
lebron_season_avgs = lebron_season_avgs.rename(columns=COLUMN_RENAME_MAP)

# Ensure only expected columns exist and are ordered correctly
EXPECTED_COLS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "awards", "season_type"
]

lebron_season_avgs = lebron_season_avgs[EXPECTED_COLS]

cols = {col: "TEXT NULL" for col in lebron_season_avgs.columns}
cols['season'] = "TEXT PRIMARY KEY"

# Create or use the DB
with SQLiteClient(db_path) as db:
    # 1) Create a table
    db.create_table(
        "lebron_season_avgs",
        cols
    )

    # Upsert dataframe
    db.upsert_dataframe(
        df=lebron_season_avgs, 
        table='lebron_season_avgs', 
        conflict_cols='season'
    )

    # Create the new table with the desired schema
    db.execute("""
        CREATE TABLE IF NOT EXISTS _lebron_season_avgs (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    # Copy data from the old table into the new one
    db.execute("""
        INSERT OR REPLACE INTO _lebron_season_avgs
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_type
        FROM lebron_season_avgs;
    """)

    # Swap tables
    db.execute("DROP TABLE lebron_season_avgs;")
    db.execute('ALTER TABLE _lebron_season_avgs RENAME TO lebron_season_avgs;')

    # Query the data
    data = db.query("SELECT * FROM lebron_season_avgs")



In [243]:
pd.DataFrame(data).head()

,season,age,team,lg,pos,g,gs,mp,fg,fga,...,drb,trb,ast,stl,blk,tov,pf,pts,awards,season_type
0,2003-04,19,CLE,NBA,SG,79,79,39.5,7.9,18.9,...,4.2,5.5,5.9,1.6,0.7,3.5,1.9,20.9,"MVP-9,ROY-1",Regular
1,2004-05,20,CLE,NBA,SF,80,80,42.4,9.9,21.1,...,6.0,7.4,7.2,2.2,0.7,3.3,1.8,27.2,"MVP-6,AS,NBA2",Regular
2,2005-06,21,CLE,NBA,SF,13,13,46.5,11.2,23.6,...,6.4,8.1,5.8,1.4,0.7,5.0,3.4,30.8,,Playoff
3,2006-07,22,CLE,NBA,SF,20,20,44.7,8.3,20.0,...,6.8,8.1,8.0,1.7,0.5,3.3,2.0,25.1,,Playoff
4,2007-08,23,CLE,NBA,SF,13,13,42.5,8.7,21.2,...,6.6,7.8,7.6,1.8,1.3,4.2,2.5,28.2,,Playoff


---

In [244]:
COLUMN_RENAME_MAP = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "awards", "season_type"
]

lebron_season_per36 = lebron_season_per36.rename(columns=COLUMN_RENAME_MAP)
lebron_season_per36 = lebron_season_per36[EXPECTED_COLS]

cols = {col: "TEXT NULL" for col in lebron_season_per36.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("lebron_season_per36", cols)

    db.upsert_dataframe(
        df=lebron_season_per36,
        table="lebron_season_per36",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _lebron_season_per36 (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _lebron_season_per36
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_type
        FROM lebron_season_per36;
    """)

    db.execute("DROP TABLE lebron_season_per36;")
    db.execute("ALTER TABLE _lebron_season_per36 RENAME TO lebron_season_per36;")

    data = db.query("SELECT * FROM lebron_season_per36;")


In [245]:
pd.DataFrame(data).head()

,season,age,team,lg,pos,g,gs,mp,fg,fga,...,drb,trb,ast,stl,blk,tov,pf,pts,awards,season_type
0,2003-04,19,CLE,NBA,SG,79,79,3122.0,7.2,17.2,...,3.8,5.0,5.4,1.5,0.7,3.1,1.7,19.1,"MVP-9,ROY-1",Regular
1,2004-05,20,CLE,NBA,SF,80,80,3388.0,8.4,17.9,...,5.1,6.2,6.1,1.9,0.6,2.8,1.6,23.1,"MVP-6,AS,NBA2",Regular
2,2005-06,21,CLE,NBA,SF,13,13,604.0,8.7,18.3,...,4.9,6.3,4.5,1.1,0.5,3.9,2.6,23.8,,Playoff
3,2006-07,22,CLE,NBA,SF,20,20,893.0,6.7,16.1,...,5.4,6.5,6.4,1.4,0.4,2.7,1.6,20.2,,Playoff
4,2007-08,23,CLE,NBA,SF,13,13,552.0,7.4,17.9,...,5.6,6.7,6.5,1.5,1.1,3.5,2.2,23.9,,Playoff


---

In [246]:
COLUMN_RENAME_MAP_SUMS = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Trp-Dbl": "trp_dbl",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS_SUMS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "trp_dbl",
    "awards", "season_type"
]

lebron_season_sums = lebron_season_sums.rename(columns=COLUMN_RENAME_MAP_SUMS)
lebron_season_sums = lebron_season_sums[EXPECTED_COLS_SUMS]

cols = {col: "TEXT NULL" for col in lebron_season_sums.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("lebron_season_sums", cols)

    db.upsert_dataframe(
        df=lebron_season_sums,
        table="lebron_season_sums",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _lebron_season_sums (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            INTEGER,
            fg            INTEGER,
            fga           INTEGER,
            fg_perc       REAL,
            three_p       INTEGER,
            three_pa      INTEGER,
            three_p_perc  REAL,
            two_p         INTEGER,
            two_pa        INTEGER,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            INTEGER,
            fta           INTEGER,
            ft_perc       REAL,
            orb           INTEGER,
            drb           INTEGER,
            trb           INTEGER,
            ast           INTEGER,
            stl           INTEGER,
            blk           INTEGER,
            tov           INTEGER,
            pf            INTEGER,
            pts           INTEGER,
            trp_dbl       INTEGER,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _lebron_season_sums
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            trp_dbl,
            awards,
            season_type
        FROM lebron_season_sums;
    """)

    db.execute("DROP TABLE lebron_season_sums;")
    db.execute("ALTER TABLE _lebron_season_sums RENAME TO lebron_season_sums;")

    data = db.query("SELECT * FROM lebron_season_sums;")


In [247]:
pd.DataFrame(data).head()

,season,age,team,lg,pos,g,gs,mp,fg,fga,...,trb,ast,stl,blk,tov,pf,pts,trp_dbl,awards,season_type
0,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,432,465,130,58,273,149,1654,0,"MVP-9,ROY-1",Regular
1,2004-05,20,CLE,NBA,SF,80,80,3388,795,1684,...,588,577,177,52,262,146,2175,4,"MVP-6,AS,NBA2",Regular
2,2005-06,21,CLE,NBA,SF,13,13,604,146,307,...,105,76,18,9,65,44,400,2,,Playoff
3,2006-07,22,CLE,NBA,SF,20,20,893,166,399,...,161,159,34,10,66,40,501,0,,Playoff
4,2007-08,23,CLE,NBA,SF,13,13,552,113,275,...,102,99,23,17,54,33,366,1,,Playoff


---

In [248]:
COLUMN_RENAME_MAP_ADV = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "PER": "per",
    "TS%": "ts_perc",
    "3PAr": "three_par",
    "FTr": "ftr",
    "ORB%": "orb_perc",
    "DRB%": "drb_perc",
    "TRB%": "trb_perc",
    "AST%": "ast_perc",
    "STL%": "stl_perc",
    "BLK%": "blk_perc",
    "TOV%": "tov_perc",
    "USG%": "usg_perc",
    "OWS": "ows",
    "DWS": "dws",
    "WS": "ws",
    "WS/48": "ws_per_48",
    "OBPM": "obpm",
    "DBPM": "dbpm",
    "BPM": "bpm",
    "VORP": "vorp",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS_ADV = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "per", "ts_perc", "three_par", "ftr",
    "orb_perc", "drb_perc", "trb_perc",
    "ast_perc", "stl_perc", "blk_perc",
    "tov_perc", "usg_perc",
    "ows", "dws", "ws", "ws_per_48",
    "obpm", "dbpm", "bpm", "vorp",
    "awards", "season_type"
]

lebron_season_advanced = lebron_season_advanced.rename(columns=COLUMN_RENAME_MAP_ADV)
lebron_season_advanced = lebron_season_advanced[EXPECTED_COLS_ADV]

cols = {col: "TEXT NULL" for col in lebron_season_advanced.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("lebron_season_advanced", cols)

    db.upsert_dataframe(
        df=lebron_season_advanced,
        table="lebron_season_advanced",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _lebron_season_advanced (
            season      TEXT,
            age         INTEGER,
            team        TEXT,
            lg          TEXT,
            pos         TEXT,
            g           INTEGER,
            gs          INTEGER,
            mp          INTEGER,
            per         REAL,
            ts_perc     REAL,
            three_par   REAL,
            ftr         REAL,
            orb_perc    REAL,
            drb_perc    REAL,
            trb_perc    REAL,
            ast_perc    REAL,
            stl_perc    REAL,
            blk_perc    REAL,
            tov_perc    REAL,
            usg_perc    REAL,
            ows         REAL,
            dws         REAL,
            ws          REAL,
            ws_per_48   REAL,
            obpm        REAL,
            dbpm        REAL,
            bpm         REAL,
            vorp        REAL,
            awards      TEXT,
            season_type TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _lebron_season_advanced
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            per,
            ts_perc,
            three_par,
            ftr,
            orb_perc,
            drb_perc,
            trb_perc,
            ast_perc,
            stl_perc,
            blk_perc,
            tov_perc,
            usg_perc,
            ows,
            dws,
            ws,
            ws_per_48,
            obpm,
            dbpm,
            bpm,
            vorp,
            awards,
            season_type
        FROM lebron_season_advanced;
    """)

    db.execute("DROP TABLE lebron_season_advanced;")
    db.execute("ALTER TABLE _lebron_season_advanced RENAME TO lebron_season_advanced;")

    data = db.query("SELECT * FROM lebron_season_advanced;")


In [249]:
pd.DataFrame(data).head()

,season,age,team,lg,pos,g,gs,mp,per,ts_perc,...,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp,awards,season_type
0,2003-04,19,CLE,NBA,SG,79,79,3122,18.3,0.488,...,2.4,2.6,5.1,0.078,2.3,-0.6,1.7,2.9,"MVP-9,ROY-1",Regular
1,2004-05,20,CLE,NBA,SF,80,80,3388,25.7,0.554,...,9.7,4.6,14.3,0.203,7.0,1.7,8.6,9.1,"MVP-6,AS,NBA2",Regular
2,2005-06,21,CLE,NBA,SF,13,13,604,23.2,0.557,...,1.0,0.7,1.7,0.135,6.3,1.2,7.5,1.4,,Playoff
3,2006-07,22,CLE,NBA,SF,20,20,893,23.9,0.516,...,2.1,1.6,3.7,0.200,5.3,2.8,8.1,2.2,,Playoff
4,2007-08,23,CLE,NBA,SF,13,13,552,24.3,0.525,...,1.0,1.2,2.2,0.187,6.7,3.4,10.1,1.7,,Playoff


---

In [33]:
career_teams = lebron_season_avgs[['Season', 'Team']]
career_teams['Season'] = career_teams['Season'].apply(lambda x: int(x[:x.find('-')]))

career_teams.head()

C:\Users\DerDo\AppData\Local\Temp\ipykernel_11324\720288711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_teams['Season'] = career_teams['Season'].apply(lambda x: int(x[:x.find('-')]))


,Season,Team
0,2003,CLE
1,2004,CLE
2,2005,CLE
3,2006,CLE
4,2007,CLE


Need to run player_season_data for 2004-2025

In [98]:
lebron_season_2004 = scraper.player_season_data("jamesle01", 2004)


No Playoff Data Found


In [105]:
lebron_season_2005 = scraper.player_season_data("jamesle01", 2005)

No Playoff Data Found


In [123]:
# lebron_season_2006 = scraper.player_season_data("jamesle01", 2006)

In [125]:
# lebron_season_2007 = scraper.player_season_data("jamesle01", 2007)

In [127]:
# lebron_season_2008 = scraper.player_season_data("jamesle01", 2008)

In [129]:
# lebron_season_2009 = scraper.player_season_data("jamesle01", 2009)

In [131]:
# lebron_season_2010 = scraper.player_season_data("jamesle01", 2010)

In [133]:
# lebron_season_2011 = scraper.player_season_data("jamesle01", 2011)

In [145]:
# lebron_season_2012 = scraper.player_season_data("jamesle01", 2012)

In [144]:
# lebron_season_2013 = scraper.player_season_data("jamesle01", 2013)

In [142]:
# lebron_season_2014 = scraper.player_season_data("jamesle01", 2014)

In [143]:
# lebron_season_2015 = scraper.player_season_data("jamesle01", 2015)

In [137]:
# lebron_season_2016 = scraper.player_season_data("jamesle01", 2016)

In [135]:
# lebron_season_2017 = scraper.player_season_data("jamesle01", 2017)

In [121]:
# lebron_season_2018 = scraper.player_season_data("jamesle01", 2018)

In [119]:
lebron_season_2019 = scraper.player_season_data("jamesle01", 2019)

No Playoff Data Found


In [118]:
# lebron_season_2020 = scraper.player_season_data("jamesle01", 2020)

In [116]:
lebron_season_2021 = scraper.player_season_data("jamesle01", 2021)

In [115]:
lebron_season_2022 = scraper.player_season_data("jamesle01", 2022)

No Playoff Data Found


In [114]:
# lebron_season_2023 = scraper.player_season_data("jamesle01", 2023)

In [112]:
lebron_season_2024 = scraper.player_season_data("jamesle01", 2024)

In [111]:
lebron_season_2025 = scraper.player_season_data("jamesle01", 2025)

In [97]:
###############################################################################
#                               Team Functions                                #
###############################################################################

### Available Functions
###
# all_team_historical_data()
# team_historical_data(team)
# team_per36_data(team, year)
# team_avg_data(team, year)
# team_season_data(team, year)
###


Need to run team_avg_data for each team in `career_teams` and each year.

In [149]:
# scraper.team_season_data('CLE', 2004)

In [250]:
lebron_team_2004 = scraper.team_avg_data('CLE', 2004)

In [251]:
lebron_team_2005 = scraper.team_avg_data('CLE', 2005)

In [252]:
lebron_team_2006 = scraper.team_avg_data('CLE', 2006)

In [253]:
lebron_team_2007 = scraper.team_avg_data('CLE', 2007)

In [254]:
lebron_team_2008 = scraper.team_avg_data('CLE', 2008)

In [255]:
lebron_team_2009 = scraper.team_avg_data('CLE', 2009)

In [256]:
lebron_team_2010 = scraper.team_avg_data('CLE', 2010)

In [257]:
lebron_team_2011 = scraper.team_avg_data('MIA', 2011)

In [258]:
lebron_team_2012 = scraper.team_avg_data('MIA', 2012)

In [259]:
lebron_team_2013 = scraper.team_avg_data('MIA', 2013)

In [260]:
lebron_team_2014 = scraper.team_avg_data('MIA', 2014)

In [261]:
lebron_team_2015 = scraper.team_avg_data('CLE', 2015)

In [262]:
lebron_team_2016 = scraper.team_avg_data('CLE', 2016)

In [263]:
lebron_team_2017 = scraper.team_avg_data('CLE', 2017)

In [264]:
lebron_team_2018 = scraper.team_avg_data('CLE', 2018)

In [265]:
lebron_team_2019 = scraper.team_avg_data('LAL', 2019)

In [266]:
lebron_team_2020 = scraper.team_avg_data('LAL', 2020)

In [267]:
lebron_team_2021 = scraper.team_avg_data('LAL', 2021)

In [268]:
lebron_team_2022 = scraper.team_avg_data('LAL', 2022)

In [269]:
lebron_team_2023 = scraper.team_avg_data('LAL', 2023)

In [270]:
lebron_team_2024 = scraper.team_avg_data('LAL', 2024)

In [271]:
lebron_team_2025 = scraper.team_avg_data('LAL', 2025)

In [277]:
tables = {
    2004: lebron_team_2004,
    2005: lebron_team_2005,
    2006: lebron_team_2006,
    2007: lebron_team_2007,
    2008: lebron_team_2008,
    2009: lebron_team_2009,
    2010: lebron_team_2010,
    2011: lebron_team_2011,
    2012: lebron_team_2012,
    2013: lebron_team_2013,
    2014: lebron_team_2014,
    2015: lebron_team_2015,
    2016: lebron_team_2016,
    2017: lebron_team_2017,
    2018: lebron_team_2018,
    2019: lebron_team_2019,
    2020: lebron_team_2020,
    2021: lebron_team_2021,
    2022: lebron_team_2022,
    2023: lebron_team_2023,
    2024: lebron_team_2024,
    2025: lebron_team_2025,
}

for year, table in tables.items():
    table = table.copy()  # avoid view mutation issues
    table["season_year"] = year


In [281]:
lebron_team_2025

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,1,Luka Dončić,25,PG,28,28,35.1,8.8,20.0,.438,...,0.9,7.2,8.1,7.5,1.6,0.4,3.7,2.4,28.2,
1,2,Austin Reaves,26,SG,73,73,34.9,6.5,14.2,.460,...,0.8,3.7,4.5,5.8,1.1,0.3,2.4,2.1,20.2,
2,3,LeBron James,40,SF,70,70,34.9,9.3,18.1,.513,...,1.0,6.8,7.8,8.2,1.0,0.6,3.7,1.4,24.4,"MVP-6,CPOY-7,AS,NBA2"
3,4,Anthony Davis,31,C,42,42,34.3,9.5,18.0,.528,...,2.8,9.0,11.9,3.4,1.3,2.1,2.2,2.0,25.7,AS
4,5,Rui Hachimura,26,PF,59,57,31.7,5.0,9.8,.509,...,1.3,3.7,5.0,1.4,0.8,0.4,0.8,1.8,13.1,
5,6,Dorian Finney-Smith,31,PF,43,20,28.8,2.8,6.4,.442,...,1.1,2.5,3.6,1.4,0.9,0.3,0.9,1.9,7.9,
6,7,D'Angelo Russell,28,PG,29,10,26.3,4.4,10.7,.415,...,0.3,2.5,2.8,4.7,0.8,0.1,1.7,1.8,12.4,
7,8,Max Christie,21,SG,46,25,25.1,2.8,6.3,.444,...,0.5,2.1,2.7,1.4,0.8,0.5,0.8,1.6,8.5,
8,9,Gabe Vincent,28,PG,72,11,21.2,2.3,5.8,.400,...,0.3,1.0,1.3,1.4,0.7,0.2,0.6,2.1,6.4,
9,10,Jaxson Hayes,24,C,56,35,19.5,2.9,4.0,.722,...,1.4,3.4,4.8,1.0,0.6,0.9,0.8,2.4,6.8,


In [283]:
for idx, table in enumerate([ lebron_team_2004, lebron_team_2005, lebron_team_2006, lebron_team_2007, lebron_team_2008, lebron_team_2009, lebron_team_2010, lebron_team_2011, lebron_team_2012, lebron_team_2013, lebron_team_2014, lebron_team_2015, lebron_team_2016, lebron_team_2017, lebron_team_2018, lebron_team_2019, lebron_team_2020, lebron_team_2021, lebron_team_2022, lebron_team_2023, lebron_team_2024, lebron_team_2025 ]): 
    year = idx + 2004 
    table['Year'] = year


In [285]:
lebron_team_data = pd.concat([
    lebron_team_2004,
    lebron_team_2005,
    lebron_team_2006,
    lebron_team_2007,
    lebron_team_2008,
    lebron_team_2009,
    lebron_team_2010,
    lebron_team_2011,
    lebron_team_2012,
    lebron_team_2013,
    lebron_team_2014,
    lebron_team_2015,
    lebron_team_2016,
    lebron_team_2017,
    lebron_team_2018,
    lebron_team_2019,
    lebron_team_2020,
    lebron_team_2021,
    lebron_team_2022,
    lebron_team_2023,
    lebron_team_2024,
    lebron_team_2025
], axis=0)

In [286]:
lebron_team_data.columns

Index(['Rk', 'Player', 'Age', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Awards',
       'Year'],
      dtype='object')

In [287]:
lebron_team_data["player_year"] = (
    lebron_team_data["Player"].astype(str)
    + "_"
    + lebron_team_data["Year"].astype(str)
)

In [236]:
with SQLiteClient(db_path) as db:
    db.execute("DROP TABLE lebron_team_data;")

In [288]:
COLUMN_RENAME_MAP_TEAM = {
    "Rk": "rk",
    "Player": "player",
    "Age": "age",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Year": "season_year",
    "player_year": "player_year"
}

EXPECTED_COLS_TEAM = [
    "rk",
    "player",
    "age",
    "pos",
    "g",
    "gs",
    "mp",
    "fg",
    "fga",
    "fg_perc",
    "three_p",
    "three_pa",
    "three_p_perc",
    "two_p",
    "two_pa",
    "two_p_perc",
    "efg_perc",
    "ft",
    "fta",
    "ft_perc",
    "orb",
    "drb",
    "trb",
    "ast",
    "stl",
    "blk",
    "tov",
    "pf",
    "pts",
    "awards",
    "season_year",
    "player_year"
]

# Rename columns
lebron_team_data = lebron_team_data.rename(columns=COLUMN_RENAME_MAP_TEAM)

# Create the composite key column (MUST exist before selecting EXPECTED_COLS_TEAM)
lebron_team_data["player_year"] = (
    lebron_team_data["player"].astype(str) + "_" + lebron_team_data["season_year"].astype(str)
)

# Enforce order
lebron_team_data = lebron_team_data[EXPECTED_COLS_TEAM]

# Staging schema
cols = {col: "TEXT NULL" for col in lebron_team_data.columns}
cols["player_year"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("lebron_team_data", cols)

    db.upsert_dataframe(
        df=lebron_team_data,
        table="lebron_team_data",
        conflict_cols="player_year"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _lebron_team_data (
            rk            INTEGER,
            player        TEXT,
            age           INTEGER,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_year   INTEGER,
            player_year   TEXT,
            PRIMARY KEY (player_year)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _lebron_team_data
        SELECT
            rk,
            player,
            age,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_year,
            player_year
        FROM lebron_team_data;
    """)

    db.execute("DROP TABLE lebron_team_data;")
    db.execute("ALTER TABLE _lebron_team_data RENAME TO lebron_team_data;")

    data = db.query("SELECT * FROM lebron_team_data;")


In [289]:
with SQLiteClient(db_path) as db:
    data = pd.DataFrame(db.query("SELECT * FROM lebron_team_data;"))

In [ ]:
pd.DataFrame(data)

,rk,player,age,pos,g,gs,mp,fg,fga,fg_perc,...,trb,ast,stl,blk,tov,pf,pts,awards,season_year,player_year
0,1,LeBron James,19,SG,79,79,39.5,7.9,18.9,0.417,...,5.5,5.9,1.6,0.7,3.5,1.9,20.9,"MVP-9,ROY-1",2004,LeBron James_2004
1,2,Ricky Davis,24,SG,22,22,36.2,6.1,14.1,0.431,...,5.5,5.0,1.1,0.4,3.1,2.3,15.3,6MOY-14,2004,Ricky Davis_2004
2,3,Jeff McInnis,29,PG,31,31,35.4,4.5,10.7,0.417,...,2.6,7.5,1.2,0.1,1.9,2.2,11.7,,2004,Jeff McInnis_2004
3,4,Carlos Boozer,22,PF,75,75,34.6,6.3,12.0,0.523,...,11.4,2.0,1.0,0.7,1.8,2.7,15.5,,2004,Carlos Boozer_2004
4,5,Zydrunas Ilgauskas,28,C,81,81,31.3,5.8,11.9,0.483,...,8.1,1.3,0.5,2.5,2.0,3.4,15.3,,2004,Zydrunas Ilgauskas_2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,20,Armel Traoré,22,SF,9,0,7.4,0.7,2.1,0.316,...,1.7,0.1,0.4,0.2,0.3,0.7,1.6,,2025,Armel Traoré_2025
427,21,Jalen Hood-Schifino,21,SG,2,0,7.0,0.5,0.5,1.0,...,0.5,0.5,0.0,0.5,0.5,1.5,2.0,,2025,Jalen Hood-Schifino_2025
428,22,Bronny James,20,SG,27,1,6.7,0.8,2.5,0.313,...,0.7,0.8,0.3,0.1,0.5,0.5,2.3,,2025,Bronny James_2025
429,23,Quincy Olivari,23,PG,2,0,5.0,0.5,2.5,0.2,...,0.0,0.5,0.0,0.0,0.5,0.0,1.5,,2025,Quincy Olivari_2025
